In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
nltk.download('averaged_perceptron_tagger')
import pandas as pd
import json

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /mnt/workspace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Apply word tokenization and part-of-speech tagging

In [3]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

## Open the json file and list the conversation

In [13]:
with open('train_data.json', 'r') as f:
    dict_json_weighted = json.load(f)
object_1 = list(dict_json_weighted.keys())[1]
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])

0-I'm looking for a train schedule after 15:15, going to Cambridge.
1-There are several trains available for you. Where are you departing from and what day would you prefer?
2-i'm leaving from broxbourne to leave on saturday after 15:15
3-Is there a time you would prefer to arrive?
4-No, but I would like the booking made for two people.
5-Booking was successful, the total fee is 35.79 GBP payable at the station .
Reference number is : N5FLMFK9. Anything else you need?
6-Yes, are there any italian restaurants in the center of town?
7-Yes I have pizza hut city centre located at regent street city centre.  Would you like to make a table reservation?
8-Yes please. Can you book it for Saturday at 19:45 for 2 and get me a reference number?
9-I'm sorry, they didn't have that time available. Would you like to try a different time or a different day?
10-Yes, can you try for 18:45 instead?
11-Sure thing! Your reference number is D1CJSO6Z.
12-Thank you for your help!
13-Did you need any further a

## define the pattern

In [5]:
#pattern = 'NP: {<DT>?<JJ>*<NN>}'
pattern_location = '~[Location]:{<from>?<NNP>|<TO>?<NNP>}'
pattern_time= '~[time]:{<at>?<CD>|<before>?<CD>|<after>?<CD>}'

## identify named entities and apply the parsing

In [15]:
object_1 = list(dict_json_weighted.keys())[1]
print(object_1)
location = 0;
time =0;
day=0;
result= open("Using_NLTK/"+object_1[:-5]+".txt","w+")
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(dict_json_weighted[object_1]['log'][i]['text'])
    sent = preprocess(dict_json_weighted[object_1]['log'][i]['text'])
    
    #print(sent)
    for j in range(len(sent)):
        if sent[j][0]=='from'  :
            if location==0: 
                result.write("~[Location]\n")
                location=1;
            #print(sent[j+1][0])
            result.write("    "+sent[j+1][0]+"\n")
            #print (sent[j])
        elif sent[j][0]=='to' and (sent[j+1][1]!='VBZ' and sent[j+1][1]!='VB' and sent[j+1][1]!='NN'):
            if location==0: 
                result.write("~[Location]\n")
                location=1;
            result.write("    "+sent[j+1][0]+"\n")
            #print (sent[j])
        elif (sent[j][0]=='at' and sent[j+1][1]=='CD') or (sent[j][0]=='after' and sent[j+1][1]=='CD'):
            if time==0: 
                result.write("~[Time]\n")
                time=1;
            result.write("    "+sent[j+1][0]+"\n")
    cp = nltk.RegexpParser(pattern_location)
    #cp = nltk.RegexpParser(pattern_time)
    cs = cp.parse(sent)
    print(cs)
    print('*****************************************')

MUL1434.json
I'm looking for a train schedule after 15:15, going to Cambridge.
(S
  I/PRP
  'm/VBP
  looking/VBG
  for/IN
  a/DT
  train/NN
  schedule/NN
  after/IN
  15:15/CD
  ,/,
  going/VBG
  (~[Location] to/TO Cambridge/NNP)
  ./.)
*****************************************
There are several trains available for you. Where are you departing from and what day would you prefer?
(S
  There/EX
  are/VBP
  several/JJ
  trains/NNS
  available/JJ
  for/IN
  you/PRP
  ./.
  Where/WRB
  are/VBP
  you/PRP
  departing/VBG
  from/IN
  and/CC
  what/WP
  day/NN
  would/MD
  you/PRP
  prefer/VB
  ?/.)
*****************************************
i'm leaving from broxbourne to leave on saturday after 15:15
(S
  i/JJ
  'm/VBP
  leaving/VBG
  from/IN
  broxbourne/NN
  to/TO
  leave/VB
  on/IN
  saturday/NN
  after/IN
  15:15/CD)
*****************************************
Is there a time you would prefer to arrive?
(S
  Is/VBZ
  there/EX
  a/DT
  time/NN
  you/PRP
  would/MD
  prefer/VB
  to/TO
  arrive